### Introduction
The objective of the project is to look for a location to set up the HQs of a new gaming company. The company will have 87 employees. We have a database with 18.800 companies (10.800 offices) with their locations that we can use as starting locations. Different groups of employees have different requests they want for the office location. We will use the foursquare API to look for locations of coffeshops, schools, recreational places, etc. and decide the optimal location for the new office. 

### Import libraries & functions and connect mongoDB

In [537]:
import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
from pymongo import MongoClient
import geopandas as gpd
import numpy as np
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from cartoframes.viz import Map as Map2, Layer, popup_element

In [538]:
import sys

sys.path.append('../src')

from geo_functions import *

In [539]:
client = MongoClient("localhost:27017")
db = client['Ironhack']
c = db.get_collection('Companies')

### We are going to filter the database to choose the appropriate city to locate the offices

Filter for companies that have raised more than 1 Million (USD or EUR)

In [540]:
filter_ = {"$and":
             [{'offices': {'$exists': 1}},
             {'total_money_raised' : {'$regex' : '[$€].*[MB]'}}]}
projection = {'name':1, '_id':0, 'total_money_raised':1, 'offices.country_code': 1, "offices.state_code":1,'offices.city':1,'offices.latitude':1,'offices.longitude':1}
list_ = list(c.find(filter_, projection).sort('offices.country_code'))[20:]

In [541]:
df = pd.DataFrame(list_).explode("offices").reset_index(drop=True)
df = pd.concat([df, df["offices"].apply(pd.Series)], axis=1).reset_index(drop=True)
df.dropna(subset=["latitude"],inplace=True)
df.dropna(subset=["city"],inplace=True)
df.drop(columns= 'offices', inplace=True)
df.drop(columns= 0, inplace=True)
df[:5]


,name,total_money_raised,city,state_code,country_code,latitude,longitude
146,Rally Software,$68.9M,Boulder,CO,USA,40.010295,-105.242005
150,Rally Software,$68.9M,Amsterdam,None,ANT,52.338098,4.868810
151,Rally Software,$68.9M,Buckinghamshire,None,GBR,51.666619,-0.614003
155,GET Holding NV,€4.25M,Zaandijk,None,NLD,52.475393,4.812833
159,Google,$555M,Mountain View,CA,USA,37.421972,-122.084143


In [542]:
df['city'].value_counts()[:5]

San Francisco    318
New York         221
Mountain View     90
Seattle           88
Palo Alto         84
Name: city, dtype: int64

Filter for companies that focus on video games

In [543]:
filter_2 = {"$and": 
             [{"category_code":"games_video"},
             {'offices': {'$exists': 1}}]}
projection_2 = {'name':1, '_id':0, 'category_code':1, 'offices.country_code': 1, "offices.state_code":1,'offices.city':1, 'offices.latitude':1,'offices.longitude':1}
list_2 = list(c.find(filter_2, projection_2).sort('offices.country_code'))[20:]

In [544]:
df_2 = pd.DataFrame(list_2).explode("offices").reset_index(drop=True)
df_2 = pd.concat([df_2, df_2["offices"].apply(pd.Series)], axis=1).reset_index(drop=True)
df_2.drop(columns= 'offices', inplace=True)
df_2.dropna(subset=["city"],inplace=True)
df_2.dropna(subset=["latitude"],inplace=True)
df_2.drop(columns= 0, inplace=True)
df_2 = df_2[df_2['city']!='']
df_2[:5]

,name,category_code,city,state_code,country_code,latitude,longitude
226,Tweegee,games_video,Ramat Gan,None,ISR,32.037363,34.800370
230,Tweegee,games_video,Ramat Gan,None,ISR,32.037363,34.800370
234,CreationFlow,games_video,Ciudad de Buenos Aires,None,ARG,-34.602858,-58.433954
239,2threads,games_video,sydney,None,AUS,-33.884685,151.216427
240,Sporting Connections,games_video,Maroochydore,None,AUS,-26.665908,153.088303


In [545]:
df_2['city'].value_counts()[:5]

New York         51
San Francisco    49
Los Angeles      25
London           20
Palo Alto        16
Name: city, dtype: int64

Filter for companies that focus on design

In [546]:
filter_3 = {"$and": 
             [{"tag_list": {'$regex': "design"}},
             {'offices': {'$exists': 1}}]}
projection_3 = {'name':1, '_id':0, 'tag_list':1, 'offices.country_code': 1, "offices.state_code":1,'offices.city':1, 'offices.latitude':1,'offices.longitude':1}
list_3 = list(c.find(filter_3, projection_3).sort('offices.country_code'))

In [547]:
df_3 = pd.DataFrame(list_3).explode("offices").reset_index(drop=True)
df_3 = pd.concat([df_3, df_3["offices"].apply(pd.Series)], axis=1).reset_index(drop=True)
df_3.drop(columns= 'offices', inplace=True)
df_3.dropna(subset=["city"],inplace=True)
df_3.dropna(subset=["latitude"],inplace=True)
df_3.drop(columns= 0, inplace=True)
df_3 = df_3[df_3['city']!='']
df_3[:5]

,name,tag_list,city,state_code,country_code,latitude,longitude
101,Making Sense,"custom-software-development, email-marketing, ...",San Antonio,TX,USA,29.516352,-98.436760
102,Making Sense,"custom-software-development, email-marketing, ...",Buenos Aires,None,ARG,-34.589281,-58.432970
103,Making Sense,"custom-software-development, email-marketing, ...",Mar del Plata,None,ARG,-38.006092,-57.558244
105,Popego,"techcrunch50, tc50, web-service, interests, se...",San Francisco,CA,USA,37.778687,-122.421242
108,Popego,"techcrunch50, tc50, web-service, interests, se...",San Francisco,CA,USA,37.778687,-122.421242


In [548]:
df_3['city'].value_counts()[:5]

New York         18
London           14
San Francisco    12
Los Angeles       9
San Diego         8
Name: city, dtype: int64

In [552]:
# Im going to focus on San Francisco
# Lots of tech start ups with +1 M funding
# Many Video games companies nearby
# Some design companies nearby

In [549]:
df_SF = df[df['city'] == 'San Francisco']

In [550]:
df_2_SF = df_2[df_2['city'] == 'San Francisco']

In [551]:
df_3_SF = df_3[df_3['city'] == 'San Francisco']

### Create a collection with the offices in the Companies DB

In [553]:
o = db.get_collection('Offices')

In [554]:
# populate the collection, selecting companies who don't have null values for coordinates and omiting
# old IDs so new IDs can de created that are unique for each office
new_collection = c.aggregate([
    {"$unwind":"$offices"},
    {"$match":{"offices.latitude":{"$ne":None}, "offices.longitude":{"$ne":None}}},
    {"$project":{"_id":0}}])
o.insert_many(new_collection)   

In [555]:
list_offices = o.find({},{"offices":1})

In [556]:
# Format geoJSON
for company in list_offices:
    geojson = {
        "type":"Point",
        "coordinates":[company["offices"]["longitude"], company["offices"]["latitude"]]
    } 
    o.update_one(company, {"$set":{"geojson":geojson}})

In [557]:
#Convert coordinates to index
o.create_index([("geojson", "2dsphere")])

'geojson_2dsphere'

### Show SanFran offices in map

Look at the offices located at San Francisco and show them in a map to see where can we locate our offices.

In [558]:
# importing data from Mongo into a pandas df
total_offices_worldwide = pd.DataFrame(list(o.find()))
total_offices_worldwide = total_offices_worldwide[['name', 'offices', 'geojson']]

In [559]:
# concatenate and clean the dataframe to have only a column for name, city, long and lat
total_offices_worldwide = pd.concat([total_offices_worldwide, total_offices_worldwide['offices'].apply(pd.Series)], axis=1).reset_index(drop=True)
total_offices_worldwide = total_offices_worldwide[['name', 'city', 'longitude', 'latitude']]
total_offices_SF = total_offices_worldwide[total_offices_worldwide['city'] == 'San Francisco']
total_offices_SF.sample()

,name,city,longitude,latitude,geojson
6309,RPX Corporation,San Francisco,-122.397298,37.794948,"{'type': 'Point', 'coordinates': [-122.397298,..."


In [561]:
#Check rows 
total_offices_SF.shape

(659, 5)

### Create a map of San Francisco

In [575]:
san_fran_map = Map(location= [37.76961,-122.4537724], zoom_start=12)
san_fran_map

### Feature Groups and Heatmaps
- Create a feature group
- Add a heatmap to feature group
- Add heatmap and feature group to SF map created before

In [576]:
# create a feature group for every San Fran Company in the dataset
sf_offices_group = folium.FeatureGroup(name= 'SanFran Offices')
# adding heatmap to feature group
HeatMap(data = total_offices_SF[['latitude', 'longitude']], radius=10, gradient={0.0: 'pink', 0.3: 'blue', 0.5: 'green',  0.7: 'yellow', 1.0: 'red'}).add_to(sf_offices_group)
# add heatmap and feature group to SF map created before
sf_offices_group.add_to(san_fran_map)

In [577]:
# create a feature group for Companies that raised +1M
df_group = folium.FeatureGroup(name= 'SanFran Companies that raised +1M')
# adding heatmap to feature group
HeatMap(data = df_SF[['latitude', 'longitude']], radius=10, gradient={'0':'Navy', '0.25':'Blue','0.5':'Green', '0.75':'Yellow','1': 'Red'}).add_to(df_group)
# add heatmap and feature group to SF map created before
df_group.add_to(san_fran_map)

In [578]:
# create a feature group for Video Games companies
df_2_group = folium.FeatureGroup(name= 'SanFran Video Games Companies')
HeatMap(data = df_2_SF[['latitude', 'longitude']], radius=15, gradient = {0.4: 'yellow', 0.65: 'orange', 1: 'red'}).add_to(df_2_group)
df_2_group.add_to(san_fran_map)

In [579]:
# create a feature group for Design companies
df_3_group = folium.FeatureGroup(name= 'SanFran Design Companies')
HeatMap(data = df_3_SF[['latitude', 'longitude']], radius=15, gradient = {0.4: 'gray', 0.65: 'black', 1: 'white'}).add_to(df_3_group)
df_3_group.add_to(san_fran_map)

In [580]:
# Create different layers
folium.LayerControl(collapsed=False, position="topleft").add_to(san_fran_map)

In [581]:
san_fran_map

### Based on the heatmap we are going to choose 6 points that are interesting for locating the offices

In [582]:
# we take the lat and long from google maps and create an icon that will be plotted on the map 
point1_lat = 37.775094283503506
point1_long = -122.41917729918815

icon = Icon(color = "red",
            opacity = 0.1,
            prefix = "fa",
            icon = "map-pin",
            icon_color = "white"
)
point1 = Marker(location = [point1_lat, point1_long], tooltip="Potential Location 1", icon=icon)
point1.add_to(san_fran_map)

In [583]:
point2_lat = 37.781292586285396
point2_long = -122.40804408884459

icon = Icon(color = "red",
            opacity = 0.1,
            prefix = "fa",
            icon = "map-pin",
            icon_color = "white"
)
point2 = Marker(location = [point2_lat, point2_long], tooltip="Potential Location 2", icon=icon)
point2.add_to(san_fran_map)

In [584]:
point3_lat = 37.78984423851833
point3_long = -122.40163489291831

icon = Icon(color = "red",
            opacity = 0.1,
            prefix = "fa",
            icon = "map-pin",
            icon_color = "white"
)
point3 = Marker(location = [point3_lat, point3_long], tooltip="Potential Location 3", icon=icon)
point3.add_to(san_fran_map)

In [585]:
point4_lat = 37.78663700410177
point4_long = -122.39816424509627

icon = Icon(color = "red",
            opacity = 0.1,
            prefix = "fa",
            icon = "map-pin",
            icon_color = "white"
)
point4 = Marker(location = [point4_lat, point4_long], tooltip="Potential Location 4", icon=icon)
point4.add_to(san_fran_map)

In [586]:
point5_lat = 37.78156089058974
point5_long = -122.39269183945797

icon = Icon(color = "red",
            opacity = 0.1,
            prefix = "fa",
            icon = "map-pin",
            icon_color = "white"
)
point5 = Marker(location = [point5_lat, point5_long], tooltip="Potential Location 5", icon=icon)
point5.add_to(san_fran_map)

In [587]:
point6_lat = 37.777761780813826
point6_long = -122.39695733186126

icon = Icon(color = "red",
            opacity = 0.1,
            prefix = "fa",
            icon = "map-pin",
            icon_color = "white"
)
point6 = Marker(location = [point6_lat, point6_long], tooltip="Potential Location 6", icon=icon)
point6.add_to(san_fran_map)

In [588]:
san_fran_map

Create a dataframe with the main potential locations for our offices

In [589]:
key_locations_ = pd.DataFrame({
'point1' : [point1_lat, point1_long],
'point2' : [point2_lat, point2_long],
'point3' : [point3_lat, point3_long],
'point4' : [point4_lat, point4_long], 
'point5' : [point5_lat, point5_long],
'point6' : [point6_lat, point6_long]}, 
index = ['latitude', 'longitude'] )

key_locations = key_locations_.transpose()
key_locations


,latitude,longitude
point1,37.775094,-122.419177
point2,37.781293,-122.408044
point3,37.789844,-122.401635
point4,37.786637,-122.398164
point5,37.781561,-122.392692
point6,37.777762,-122.396957


### Download data from foursquare API

In [590]:
load_dotenv()
token_fsq = os.getenv("token_foursquare")

We now need to dowload data from foursquare so we can identify locations of places we want to know

In [591]:
#Starbucks--> we will use a query
#parks = '16032'
#airports = '19031'
#dog_salon--> we will use a query
#night_club = '10032'
#basketball = '18006'
#school = '12057'
#rail_station = '19047'
#port = '19023'
#vegan = '13377'

In [592]:
# This function gets a query, the latitude and longitude from the API and returns a list with the distance from the point.
def foursquare_query (query, lat, lon):

    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&limit=1"

    headers = {"accept": "application/json", "Authorization": token_fsq}
    response = requests.get(url, headers=headers).json()

    list = []
    for i in response["results"]:
        distance = i["distance"]

        list.append(distance)
    
    return list

In [593]:
#This function gets a category, the latitude and longitude from the API and returns a list with the distance from the point.
def foursquare_cat (category, lat, lon):

    url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{lon}&categories={category}&limit=1"

    headers = {"accept": "application/json", "Authorization": token_fsq}
    response = requests.get(url, headers=headers).json()

    list = []
    for i in response["results"]:
        distance = i["distance"]

        list.append(distance)
    
    return list

For each query or category 

In [486]:
query = 'Starbucks' #Starbucks
starbucks=[]
for i in zip(key_locations["latitude"],key_locations["longitude"]):
    starbucks.append(foursquare_query(query,i[0],i[1]))
key_locations["Nearest Starbucks (m)"] = starbucks
key_locations

,latitude,longitude,Nearest Starbucks (m)
point1,37.775094,-122.419177,[237]
point2,37.781293,-122.408044,[319]
point3,37.789844,-122.401635,[724]
point4,37.786637,-122.398164,[435]
point5,37.781561,-122.392692,[747]
point6,37.777762,-122.396957,[1200]


In [488]:
category = 16032 #Outdoor Parks
park=[]
for i in zip(key_locations["latitude"],key_locations["longitude"]):
    park.append(foursquare_cat(category,i[0],i[1]))
key_locations["Nearest park (m)"] = park
key_locations

,latitude,longitude,Nearest Starbucks (m),Nearest park (m)
point1,37.775094,-122.419177,[237],[1994]
point2,37.781293,-122.408044,[319],[620]
point3,37.789844,-122.401635,[724],[959]
point4,37.786637,-122.398164,[435],[435]
point5,37.781561,-122.392692,[747],[937]
point6,37.777762,-122.396957,[1200],[924]


In [489]:
category = 19031 #Airports
airport =[]
for i in zip(key_locations["latitude"],key_locations["longitude"]):
    airport.append(foursquare_cat(category,i[0],i[1]))
key_locations["Nearest Airport (m)"] = airport
key_locations

,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m)
point1,37.775094,-122.419177,[237],[1994],[17715]
point2,37.781293,-122.408044,[319],[620],[18275]
point3,37.789844,-122.401635,[724],[959],[19177]
point4,37.786637,-122.398164,[435],[435],[18804]
point5,37.781561,-122.392692,[747],[937],[18224]
point6,37.777762,-122.396957,[1200],[924],[17814]


In [492]:
#Barbershop ¿for dogs?, doing it one by one because using the id for pets grooming gave wrong results
dog_salon = []

dog_salon.append(foursquare_query('dog barber', 37.776745, -122.417106))
dog_salon.append(foursquare_query('Doggie Day Spaw', 37.781293, -122.408044))
dog_salon.append(foursquare_query('Alpha Gromming Pet Salon', 37.789844, -122.401635))
dog_salon.append(foursquare_query('Doggie Day Spaw', 37.786637, -122.398164))	
dog_salon.append(foursquare_query('Doggie Day Spaw', 37.781561, -122.392692))
dog_salon.append(foursquare_query('Doggie Day Spaw', 37.777762, -122.396957))	

key_locations["Nearest Dog Salon (m)"] = dog_salon
key_locations


,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m)
point1,37.775094,-122.419177,[237],[1994],[17715],[1015]
point2,37.781293,-122.408044,[319],[620],[18275],[556]
point3,37.789844,-122.401635,[724],[959],[19177],[409]
point4,37.786637,-122.398164,[435],[435],[18804],[953]
point5,37.781561,-122.392692,[747],[937],[18224],[919]
point6,37.777762,-122.396957,[1200],[924],[17814],[504]


In [493]:
query = 'night club' #Night Clubs
night_clubs =[]
for i in zip(key_locations["latitude"],key_locations["longitude"]):
    night_clubs.append(foursquare_query(query,i[0],i[1]))
key_locations["Nearest Night Clubs (m)"] = night_clubs
key_locations

,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m),Nearest Night Clubs (m)
point1,37.775094,-122.419177,[237],[1994],[17715],[1015],[1176]
point2,37.781293,-122.408044,[319],[620],[18275],[556],[1541]
point3,37.789844,-122.401635,[724],[959],[19177],[409],[2520]
point4,37.786637,-122.398164,[435],[435],[18804],[953],[2248]
point5,37.781561,-122.392692,[747],[937],[18224],[919],[1974]
point6,37.777762,-122.396957,[1200],[924],[17814],[504],[1412]


In [494]:
category = 18006 #Basketball
basketball =[]
for i in zip(key_locations["latitude"],key_locations["longitude"]):
    basketball.append(foursquare_cat(category,i[0],i[1]))
key_locations["Nearest Basketball Stadium (m)"] = basketball
key_locations

,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m),Nearest Night Clubs (m),Nearest Basketball Stadium (m)
point1,37.775094,-122.419177,[237],[1994],[17715],[1015],[1176],[1899]
point2,37.781293,-122.408044,[319],[620],[18275],[556],[1541],[1369]
point3,37.789844,-122.401635,[724],[959],[19177],[409],[2520],[2019]
point4,37.786637,-122.398164,[435],[435],[18804],[953],[2248],[1636]
point5,37.781561,-122.392692,[747],[937],[18224],[919],[1974],[1166]
point6,37.777762,-122.396957,[1200],[924],[17814],[504],[1412],[655]


In [495]:
query = 'school' #Schools
school =[]
for i in zip(key_locations["latitude"],key_locations["longitude"]):
    school.append(foursquare_query(query,i[0],i[1]))
key_locations["Nearest School (m)"] = school
key_locations

,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m),Nearest Night Clubs (m),Nearest Basketball Stadium (m),Nearest School (m)
point1,37.775094,-122.419177,[237],[1994],[17715],[1015],[1176],[1899],[531]
point2,37.781293,-122.408044,[319],[620],[18275],[556],[1541],[1369],[1019]
point3,37.789844,-122.401635,[724],[959],[19177],[409],[2520],[2019],[2124]
point4,37.786637,-122.398164,[435],[435],[18804],[953],[2248],[1636],[2014]
point5,37.781561,-122.392692,[747],[937],[18224],[919],[1974],[1166],[2056]
point6,37.777762,-122.396957,[1200],[924],[17814],[504],[1412],[655],[1545]


In [496]:
category = 19047 #Rail Station
rail_station =[]
for i in zip(key_locations["latitude"],key_locations["longitude"]):
    rail_station.append(foursquare_cat(category,i[0],i[1]))
key_locations["Nearest Rail Station (m)"] = rail_station
key_locations

,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m),Nearest Night Clubs (m),Nearest Basketball Stadium (m),Nearest School (m),Nearest Rail Station (m)
point1,37.775094,-122.419177,[237],[1994],[17715],[1015],[1176],[1899],[531],[22]
point2,37.781293,-122.408044,[319],[620],[18275],[556],[1541],[1369],[1019],[311]
point3,37.789844,-122.401635,[724],[959],[19177],[409],[2520],[2019],[2124],[550]
point4,37.786637,-122.398164,[435],[435],[18804],[953],[2248],[1636],[2014],[727]
point5,37.781561,-122.392692,[747],[937],[18224],[919],[1974],[1166],[2056],[619]
point6,37.777762,-122.396957,[1200],[924],[17814],[504],[1412],[655],[1545],[229]


In [497]:
category = 19023 #Port
port =[]
for i in zip(key_locations["latitude"],key_locations["longitude"]):
    port.append(foursquare_cat(category,i[0],i[1]))
key_locations["Nearest Port (m)"] = port
key_locations


,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m),Nearest Night Clubs (m),Nearest Basketball Stadium (m),Nearest School (m),Nearest Rail Station (m),Nearest Port (m)
point1,37.775094,-122.419177,[237],[1994],[17715],[1015],[1176],[1899],[531],[22],[3203]
point2,37.781293,-122.408044,[319],[620],[18275],[556],[1541],[1369],[1019],[311],[2057]
point3,37.789844,-122.401635,[724],[959],[19177],[409],[2520],[2019],[2124],[550],[956]
point4,37.786637,-122.398164,[435],[435],[18804],[953],[2248],[1636],[2014],[727],[1148]
point5,37.781561,-122.392692,[747],[937],[18224],[919],[1974],[1166],[2056],[619],[1689]
point6,37.777762,-122.396957,[1200],[924],[17814],[504],[1412],[655],[1545],[229],[2107]


In [498]:
category = 13377 #Vegan Restaurant
vegan =[]
for i in zip(key_locations["latitude"],key_locations["longitude"]):
    vegan.append(foursquare_cat(category,i[0],i[1]))
key_locations["Nearest Vegan Restaurant (m)"] = vegan
key_locations

,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m),Nearest Night Clubs (m),Nearest Basketball Stadium (m),Nearest School (m),Nearest Rail Station (m),Nearest Port (m),Nearest Vegan Restaurant (m)
point1,37.775094,-122.419177,[237],[1994],[17715],[1015],[1176],[1899],[531],[22],[3203],[1250]
point2,37.781293,-122.408044,[319],[620],[18275],[556],[1541],[1369],[1019],[311],[2057],[983]
point3,37.789844,-122.401635,[724],[959],[19177],[409],[2520],[2019],[2124],[550],[956],[389]
point4,37.786637,-122.398164,[435],[435],[18804],[953],[2248],[1636],[2014],[727],[1148],[80]
point5,37.781561,-122.392692,[747],[937],[18224],[919],[1974],[1166],[2056],[619],[1689],[820]
point6,37.777762,-122.396957,[1200],[924],[17814],[504],[1412],[655],[1545],[229],[2107],[1065]


In [499]:
key_locations["Nearest Starbucks (m)"] = key_locations["Nearest Starbucks (m)"].apply(lambda x: x[0])
key_locations["Nearest park (m)"] = key_locations["Nearest park (m)"].apply(lambda x: x[0])
key_locations["Nearest Airport (m)"] = key_locations["Nearest Airport (m)"].apply(lambda x: x[0])
key_locations["Nearest Dog Salon (m)"] = key_locations["Nearest Dog Salon (m)"].apply(lambda x: x[0])
key_locations["Nearest Night Clubs (m)"] = key_locations["Nearest Night Clubs (m)"].apply(lambda x: x[0])
key_locations["Nearest Basketball Stadium (m)"] = key_locations["Nearest Basketball Stadium (m)"].apply(lambda x: x[0])
key_locations["Nearest School (m)"] = key_locations["Nearest School (m)"].apply(lambda x: x[0])
key_locations["Nearest Rail Station (m)"] = key_locations["Nearest Rail Station (m)"].apply(lambda x: x[0])
key_locations["Nearest Port (m)"] = key_locations["Nearest Port (m)"].apply(lambda x: x[0])
key_locations["Nearest Vegan Restaurant (m)"] = key_locations["Nearest Vegan Restaurant (m)"].apply(lambda x: x[0])

In [500]:
key_locations

,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m),Nearest Night Clubs (m),Nearest Basketball Stadium (m),Nearest School (m),Nearest Rail Station (m),Nearest Port (m),Nearest Vegan Restaurant (m)
point1,37.775094,-122.419177,237,1994,17715,1015,1176,1899,531,22,3203,1250
point2,37.781293,-122.408044,319,620,18275,556,1541,1369,1019,311,2057,983
point3,37.789844,-122.401635,724,959,19177,409,2520,2019,2124,550,956,389
point4,37.786637,-122.398164,435,435,18804,953,2248,1636,2014,727,1148,80
point5,37.781561,-122.392692,747,937,18224,919,1974,1166,2056,619,1689,820
point6,37.777762,-122.396957,1200,924,17814,504,1412,655,1545,229,2107,1065


In [437]:
### Weight
#Leisure
#Starbucks = 0.15
#Night Clubs = 0.1
#Vegan Rest = 0.04

#Kids
#Parks = 0.1
#School = 0.15

#Travel
#Airport = 0.25
#Rail Station = 0.075
#Port = 0.075

#Others
#Basketball St = 0.03
#Dog Salon = 0.03

In [503]:
key_locations['Score'] = ((100/key_locations['Nearest Starbucks (m)']*0.15) *100) + ((100/key_locations['Nearest Night Clubs (m)']*0.1)*100) + ((100/key_locations['Nearest Vegan Restaurant (m)']*0.04)*100) + ((100/key_locations['Nearest park (m)']*0.1)*100) + ((100/key_locations['Nearest School (m)']*0.15)*100) + ((100/key_locations['Nearest Airport (m)']*0.25)*100) + ((100/key_locations['Nearest Rail Station (m)']*0.075)*100) + ((100/key_locations['Nearest Port (m)']*0.075)*100) + ((100/key_locations['Nearest Basketball Stadium (m)']*0.03)*100) + ((100/key_locations['Nearest Dog Salon (m)']*0.03)*100)

In [504]:
key_locations

,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m),Nearest Night Clubs (m),Nearest Basketball Stadium (m),Nearest School (m),Nearest Rail Station (m),Nearest Port (m),Nearest Vegan Restaurant (m),Score
point1,37.775094,-122.419177,237,1994,17715,1015,1176,1899,531,22,3203,1250,45.745550
point2,37.781293,-122.408044,319,620,18275,556,1541,1369,1019,311,2057,983,12.514665
point3,37.789844,-122.401635,724,959,19177,409,2520,2019,2124,550,956,389,8.406498
point4,37.786637,-122.398164,435,435,18804,953,2248,1636,2014,727,1148,80,14.252820
point5,37.781561,-122.392692,747,937,18224,919,1974,1166,2056,619,1689,820,7.175825
point6,37.777762,-122.396957,1200,924,17814,504,1412,655,1545,229,2107,1065,9.211585


In [505]:
gdf_1 = gpd.GeoDataFrame(key_locations, geometry=gpd.points_from_xy(key_locations["longitude"], key_locations["latitude"]))
gdf_1

,latitude,longitude,Nearest Starbucks (m),Nearest park (m),Nearest Airport (m),Nearest Dog Salon (m),Nearest Night Clubs (m),Nearest Basketball Stadium (m),Nearest School (m),Nearest Rail Station (m),Nearest Port (m),Nearest Vegan Restaurant (m),Score,geometry
point1,37.775094,-122.419177,237,1994,17715,1015,1176,1899,531,22,3203,1250,45.745550,POINT (-122.41918 37.77509)
point2,37.781293,-122.408044,319,620,18275,556,1541,1369,1019,311,2057,983,12.514665,POINT (-122.40804 37.78129)
point3,37.789844,-122.401635,724,959,19177,409,2520,2019,2124,550,956,389,8.406498,POINT (-122.40163 37.78984)
point4,37.786637,-122.398164,435,435,18804,953,2248,1636,2014,727,1148,80,14.252820,POINT (-122.39816 37.78664)
point5,37.781561,-122.392692,747,937,18224,919,1974,1166,2056,619,1689,820,7.175825,POINT (-122.39269 37.78156)
point6,37.777762,-122.396957,1200,924,17814,504,1412,655,1545,229,2107,1065,9.211585,POINT (-122.39696 37.77776)


In [536]:
from cartoframes.viz import size_continuous_style, size_continuous_legend
Map2(Layer(gdf_1, size_continuous_style('Score', stroke_color='black', stroke_width = 2, opacity=0.2), legends=size_continuous_legend(
        title='Evaluation of places to locate the office',
        description='Weighted Average')))